# 上海期货交易所

上海期货交易所张江机房一共有 12 幢楼。其中 1 号楼目前是撮合中心，9～12 号楼均为托管机房。各个期货公司的托管服务器大多分布在这些机房内，也有小部分在其他楼里。根据各方面信息汇总，主要有以下信息：
* 一般大的期货公司会有一个独占机房（甚至两个），小的期货公司可能多个合用一个机房（每个期货公司占用几个机柜）；
* 对每个期货公司，交易所会发两根千兆网线进行接入（下图的橘色线），一主一备，两根线实际上只有一根能够使用。期货公司所有的行情、订单消息都是通过该网线进行传输。由于带宽只有千兆，因此这里可能是一个潜在瓶颈，容易造成阻塞；
* 在交易所的千兆网线接入后，券商内部的局域网一般都是万兆网络，各种行情、交易柜台服务器、客户的托管服务器等，都在该局域网中；

交易所的端的网络拓扑没有公开信息，以下信息都是猜想的：
* 千兆网线在交易所端，推断是连接到到千兆-万兆汇聚交换机，应该有多个汇聚交换机（至少一幢楼要有一个，很可能一层有一个）；
* 千兆-万兆汇聚交换机通过光纤（10G 或 40G）连接到交易所核心交换机；
* 核心交换机上除了下层的交换机外，还有交易所撮合服务器、行情切片机、排队机、交易所网关（防火墙）、以及12个前置机；
* 交易所内部核心网络是10G还是40G也不确定；

预计在2019年3月，交易所会推送新的组播行情，届时行情发布的过程会有较大变化；

## 交易所行情发布

交易所行情发布的顺序如下（交易所部分也是推断的）

* 行情切片机每隔500ms，向撮合服务器查询一个行情切片，然后将切片内容发送给12个前置机（发送方式是TCP发送，还是组播不清楚）；
* 券商1个席位可以向没个前置机各订阅一路行情（每路连接均为TCP连接），因此一个席位最多可以同时订阅12路行情；
* 由于行情是通过TCP发送的，因此如果同一个前置上有很多订阅者，这些TCP包也都是依次发送的，行情到达会有很大的先后差异。因此，券商往往会使用很多席位，订阅很多路行情；
* 每个TCP包包含了多个合约的增量行情。对交易所TCP包的分析见SHFE WireShark抓包数据分析；
* 券商的行情柜台在接收到这些TCP包后，会进行解析然后发送给客户，XELE、盛立等柜台会采用FPGA进行解析，甚至有的交换机内置了FPGA模块，直接在交换机处解析行情并发送给客户；

## 订单请求与回报流程

排队机 - 交易所用来对所有的订单请求进行排队，并设置一个统一的订单序号的服务

撮合引擎 - 在排队机对订单先后次序排序后，计算订单成交数量/价格的系统（或者加到队列中而未成交）

fens - 在柜台前置连接交易前置时，决定分配连接到哪个交易前置（柜台机也可不按照该决定连接）

交易前置 - 在交易所防火墙后，供各个券商的柜台连接的前置机

柜台前置 - 券商的客户连接的交易服务器，客户的指令均由该服务器进行转换，再发送到交易前置

席位 - 券商在交易所交易的账号，一般一个席位对应一个柜台，同一个席位的账号只能在一处登录

委托回报 - 订单在排队机确定好队列位置（即订单号）后，发送回来的确认回报

成交回报 - 对于成交的订单，由撮合引擎发送回来的成交确认消息

行情柜台 - 券商用于接收交易所行情，并向客户转发行情的服务器

订单请求

* 客户托管服务器发送请求到相应的交易柜台；
* 柜台对订单进行参数检查、持仓和保证金等检查，如果通过，则发送到连接的席位；
* 柜台与交易所前置机连接是在开盘前建立的（TCP连接），一般一个交易日内（或者至少日盘和夜盘）是不会发生变化的。但是一个柜台也有可能同时连接了多个席位，分布在不同前置机上；
* 柜台发送给前置机的订单要先经过网关、再到前置机；
* 交易所前置机在接收到订单后，也要进行一些参数检查，然后发送给排队机；
* （推断）订单队列顺序是在排队机这里完全确定的，然后订单从排队机一次进入撮合服务器进行交易；
* 目前上期所有的合约都在同一个排队机上进行排序，然后发送给撮合服务器（一台还是多台不确定，但由于订单次序是在排队机决定的，撮合服务器的数量就无关紧要了）；

订单回报

* 撮合机将订单回报发送到相应的前置机；
* 前置机发送给券商柜台，下行可能不需要经过网关了；
* 券商柜台再推送给客户服务器；

交易所共有12个交易前置，每个通过光纤连到排队机中。排队机是单台服务器，排队机与撮合服务器是否同一台物理机不确定。

每个柜台前置每天开盘前，会连接到12台交易前置的一台。然后由交易前置转发到排队机。具体路由顺序如下：

- 券商柜台通过万兆千兆光纤发送到交易所网络，进入防火墙（40GB），此处所有券商的消息流第一次被序列化，由于交换机延迟很低，因此，此处队列的顺序主要由各个券商柜台发送消息的先后顺序决定；

- 通过防火墙的消息被转发给12个交易前置机中，在每个交易前置中由于是TCP连接，这里会产生一次消息的随机化（同一个券商柜台即TCP连接发送的消息顺序不会改变，不同柜台的消息顺序会随机化）；

- 前置机处理完，转发给排队机，在排队机内消息顺序还有一次随机化（同一个前置机发送的消息顺序应该不会改变了）；

- 排队机排队后，将订单发送给撮合机，同时发送委托回报给客户。撮合服务器如果发生成交，将成交信息依次发送给相应的客户；成交回报与委托回报分别是由撮合服务器，排队机独立发送的。


分析：

对于应该会立即成交的订单，可以通过观察成交回报的返回时间与委托回报的返回时间差，来推测排队机在排队后至撮合引擎的完成撮合的时间，进而反推出排队机完成排队需要的时间；

TCP连接可能在路由器中有老化的机制：即如果较长时间没有通讯，那么在路由器内该连接的资源会被回收，下一次通讯时再次打开。可以通过交易记录分析是否存在这种老化现象（由于柜台还有其他用户，因此需要使用移动机房的ctp2分析，该柜台目前只有我们使用）；
主要的瓶颈在于第一次随机化与第二次随机化，需要通过测试确定随机化合适的模型；

【行情】

交易所共有12个行情前置。交易所的行情采集机通过某种方式向这12个行情前置发送行情数据，很可能是组播、广播，但也不排除是普通的TCP连接。同时交易所使用某种随机算法，使12个行情前置收到行情的时间大致保持一致。

行情前置与各个券商的行情柜台通过TCP连接。因此一个行情前置可能同时连接非常多的行情柜台。每个行情柜台可以连接任意多个行情前置，但是一个席位（一般对应一个柜台）同时只能连接一个行情前置一次（也就是可以同时有12个TCP连接）。

目前艾科郎克同时对12个行情前置有100多个连接，然后发送给客户最快的那个。具体分析见《SHFE行情速度分析》
 

柜台

TCP连接数量

盛立36

UDP30

飞马12

量投12

【其他】

大连交易所的客户API连接有一种冷/热效应：当连续下单后，后续的订单回报较快。如果较长一段时间没有下单，则后续订单回报较慢。这可能是由于API连接采用了某种动态分配算法，活跃的TCP连接拥有更高的权限导致的。对此，可以利用不计入订单撤销的FOK/FAK订单来保持连接处于活跃状态。上期是否有此种效应还未知。
参与测试恒生的柜台系统，这样如果有2个账号同时交易，可以获得更好的队列位置。
4月～10月SHFE的延迟增加（4ms->10ms）问题的原因还需要研究